In [417]:
import numpy as np
import pandas as pd
 
import matplotlib.pyplot as plt

import json

import re

import pickle

from sklearn.model_selection import train_test_split

from imblearn.over_sampling import RandomOverSampler

import nltk
from nltk.corpus import stopwords

from pymystem3 import Mystem

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [418]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/artemiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/artemiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [419]:
DATASET_PATH = './data/data.json'

In [420]:
with open(DATASET_PATH, 'r') as file:
    data = json.load(file)

max_length = max([len(data[elem]) for elem in data])

for elem in data:
    while len(data[elem]) < max_length:
        data[elem].append('')
        
max_length

1500

In [421]:
data = json.dumps(data)

In [422]:
df = pd.read_json(data)

df

/var/folders/m1/7thbrsg96m1_wm0p48jm8d7c0000gn/T/ipykernel_5429/792015447.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(data)


,АВТО БАТЯ,АЭРОФЛОТ,Телеграмма РЖД
0,Зачем переплачивать за доставку\n\n[АВТО БАТЯ]...,,
1,«Сравни.ру» огласил победителя ежегодной преми...,,
2,В Нигерии толпа собралась посмотреть на горящу...,,
3,"**Фургон задавил челябинку, которая отвлеклась...",,
4,[МФЦ](https://t.me/+wk0wdjBDPJJmMWMy) пришел в...,**Аэрофлот — партнёр шоу «Восхождение» Фонда С...,**Интегрировали станцию Кутузовская МЦД-4 с ме...
...,...,...,...
1495,**Десятилетний мальчик скатился с горки прямо ...,**В гостях у Аэрофлота**\n\n👧🧒🏻Накануне Дня за...,"🚆 **Перевели «на цифру» ст. Апрелевка, конечну..."
1496,Жуткая ситуация на дороге\n\n[АВТО БАТЯ](https...,**✈️ В Сочи из Самары!\n**\nЗапускаем с 1 июня...,**Погрузка экспортных грузов **[**в порты**](h...
1497,Волшебный бардачок\n\n[АВТО БАТЯ](https://t.me...,**✈️ Июньский номер журнала «Аэрофлот» - уже н...,👨‍🦽👩‍🦯 **Вокзалы для каждого: создаём инклюзив...
1498,"**«Давай эту тоже вперёд толкнём, пацаны»**\n\...",**✈️🌍 Как путешествовать по миру сегодня? \n\...,


In [423]:
df = df.melt(var_name='channel_name', value_name='post_text')

In [424]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   channel_name  4500 non-null   object
 1   post_text     4500 non-null   object
dtypes: object(2)
memory usage: 70.4+ KB


In [425]:
all_classes = df['channel_name'].unique()

all_classes

array(['АВТО БАТЯ', 'АЭРОФЛОТ', 'Телеграмма РЖД'], dtype=object)

In [426]:
NUM_WORDS = 5_000                                 # Максимальное количество слов
MAX_NEWS_LEN = 100                                # Максимальная длина новости
NB_CLASSES = all_classes.shape[0]                 # Количество классов

In [427]:
def remove_special_characters(text):
    text = re.sub(r'(http|https|www)S+', '', text) 
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'http\S+', '', text)
        
    return text

In [428]:
bad_words = ['joinchat', 'ru', 'me', 's', 't', 'vk', 'wk', 'telegram', 'xytni', 'vvd', 'www', *df['channel_name'].unique()]
words_to_remove = [lower_name.lower() for lower_name in bad_words]

def filter_text(text):
    for word in words_to_remove:
        text = text.replace(word, '')
        
    return text

In [429]:
df.replace("", pd.NA, inplace=True)
df.dropna(inplace=True)

# Преобразование текста в нижний регистр
df['post_text'] = df['post_text'].apply(lambda x: x.lower())

# Удаление специальных символов и цифр
df['post_text'] = df['post_text'].apply(remove_special_characters)

df['post_text'] = df['post_text'].apply(filter_text)

# Токенизация текста
df['tokens'] = df['post_text'].apply(nltk.word_tokenize)

# Удаление стоп-слов
stop_words = set(stopwords.words('russian'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

# Лемматизация текста
# Не забудьте установить пакеты для лемматизации, например pymystem3

m = Mystem()
df['tokens'] = df['tokens'].apply(lambda x: [m.lemmatize(word)[0] for word in x])

# Преобразование токенов обратно в текст
df['processed_text'] = df['tokens'].apply(lambda x: ' '.join(x))
# Вы можете добавить другие шаги preprocessing текста в зависимости от ваших потребностей

# В итоговом датафрейме будет колонка 'processed_text' с обработанным текстом

In [430]:
df

,channel_name,post_text,tokens,processed_text
0,АВТО БАТЯ,зачем переплачивать за доставку m,"[переплачивать, доставка, m]",переплачивать доставка m
1,АВТО БАТЯ,сравни ру огласил победителя ежегодной премии...,"[сравнивать, ру, оглашать, победитель, ежегодн...",сравнивать ру оглашать победитель ежегодный пр...
2,АВТО БАТЯ,в нигерии толпа собралась посмотреть на горящу...,"[нигерия, толпа, собираться, посмотреть, горет...",нигерия толпа собираться посмотреть гореть зап...
3,АВТО БАТЯ,фургон задавил челябинку которая отвлеклась н...,"[фургон, задавливать, челябинка, который, отвл...",фургон задавливать челябинка который отвлекать...
4,АВТО БАТЯ,мфц wk wdjbdpjjmmwmy пришел в telegram мног...,"[мфц, wk, wdjbdpjjmmwmy, приходить, telegram, ...",мфц wk wdjbdpjjmmwmy приходить telegram многоф...
...,...,...,...,...
4493,Телеграмма РЖД,лучшие из лучших определятся в честной борьбе...,"[хороший, хороший, определяться, честный, борь...",хороший хороший определяться честный борьба ек...
4494,Телеграмма РЖД,с начала года обновили электрички в регионах ...,"[начинать, год, обновлять, электричка, регион,...",начинать год обновлять электричка регион конец...
4495,Телеграмма РЖД,перевели на цифру ст апрелевка конечную остан...,"[переводить, цифра, ст, апрелевка, конечный, о...",переводить цифра ст апрелевка конечный останов...
4496,Телеграмма РЖД,погрузка экспортных грузов в порты company r...,"[погрузка, экспортный, груз, порт, company, rz...",погрузка экспортный груз порт company rzd page...


In [431]:
df_encoded = pd.get_dummies(df['channel_name'], dtype=int)

# Объединение закодированных колонок с исходным датафреймом
df = pd.concat([df, df_encoded], axis=1)

# Удаление исходной колонки 'channel_name'
df.drop('channel_name', axis=1, inplace=True)

df

,post_text,tokens,processed_text,АВТО БАТЯ,АЭРОФЛОТ,Телеграмма РЖД
0,зачем переплачивать за доставку m,"[переплачивать, доставка, m]",переплачивать доставка m,1,0,0
1,сравни ру огласил победителя ежегодной премии...,"[сравнивать, ру, оглашать, победитель, ежегодн...",сравнивать ру оглашать победитель ежегодный пр...,1,0,0
2,в нигерии толпа собралась посмотреть на горящу...,"[нигерия, толпа, собираться, посмотреть, горет...",нигерия толпа собираться посмотреть гореть зап...,1,0,0
3,фургон задавил челябинку которая отвлеклась н...,"[фургон, задавливать, челябинка, который, отвл...",фургон задавливать челябинка который отвлекать...,1,0,0
4,мфц wk wdjbdpjjmmwmy пришел в telegram мног...,"[мфц, wk, wdjbdpjjmmwmy, приходить, telegram, ...",мфц wk wdjbdpjjmmwmy приходить telegram многоф...,1,0,0
...,...,...,...,...,...,...
4493,лучшие из лучших определятся в честной борьбе...,"[хороший, хороший, определяться, честный, борь...",хороший хороший определяться честный борьба ек...,0,0,1
4494,с начала года обновили электрички в регионах ...,"[начинать, год, обновлять, электричка, регион,...",начинать год обновлять электричка регион конец...,0,0,1
4495,перевели на цифру ст апрелевка конечную остан...,"[переводить, цифра, ст, апрелевка, конечный, о...",переводить цифра ст апрелевка конечный останов...,0,0,1
4496,погрузка экспортных грузов в порты company r...,"[погрузка, экспортный, груз, порт, company, rz...",погрузка экспортный груз порт company rzd page...,0,0,1


In [432]:
df.processed_text.loc[4]

'мфц wk wdjbdpjjmmwmy приходить telegram многофункциональный центр wk wdjbdpjjmmwmy это центр просмотр описание порядок предоставление государственный услуга рассказывать какой услуга государство мочь получать совершенно бесплатно какой выплата полагать закон сэкономить налог wk wdjbdpjjmmwmy телегр канал мфц wk wdjbdpjjmmwmy подписываться wk wdjbdpjjmmwmy'

In [434]:
tokenizer = Tokenizer(num_words=NUM_WORDS)

In [435]:
tokenizer.fit_on_texts(df['processed_text'])

In [436]:
tokenizer.word_index

{'год': 1,
 'm': 2,
 'москва': 3,
 'aeroflot': 4,
 'поезд': 5,
 'который': 6,
 'ржд': 7,
 'пассажир': 8,
 'рейс': 9,
 'новый': 10,
 'наш': 11,
 'это': 12,
 'билет': 13,
 'млн': 14,
 'россия': 15,
 'тонна': 16,
 'время': 17,
 'utm': 18,
 'водитель': 19,
 'дорога': 20,
 'также': 21,
 'день': 22,
 'работа': 23,
 'место': 24,
 'сайт': 25,
 'путь': 26,
 'b': 27,
 'направление': 28,
 'перевозка': 29,
 'тыс': 30,
 'первый': 31,
 'станция': 32,
 'движение': 33,
 'полет': 34,
 'app': 35,
 'петербург': 36,
 'быть': 37,
 'область': 38,
 'автомобиль': 39,
 'путешествие': 40,
 'самый': 41,
 'один': 42,
 'earch': 43,
 'санкт': 44,
 'становиться': 45,
 'человек': 46,
 'вагон': 47,
 'route': 48,
 'cabin': 49,
 'adult': 50,
 'children': 51,
 'infant': 52,
 'железный': 53,
 'u': 54,
 'economy': 55,
 'летний': 56,
 'свой': 57,
 'сегодня': 58,
 'вокзал': 59,
 'железнодорожный': 60,
 'груз': 61,
 'раз': 62,
 'mow': 63,
 'компания': 64,
 'получать': 65,
 'сеть': 66,
 'город': 67,
 'маршрут': 68,
 'машина': 

In [437]:
for channel_name in all_classes:
    print(f'{channel_name} in tokenizer is', channel_name in tokenizer.word_index)

АВТО БАТЯ in tokenizer is  False
АЭРОФЛОТ in tokenizer is  False
Телеграмма РЖД in tokenizer is  False


In [438]:
TOKENIZER_PATH = './learned_models'

with open(f'{TOKENIZER_PATH}/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [439]:
sequences = tokenizer.texts_to_sequences(df['processed_text'])

In [440]:
EXAMPLE_INDEX = 1

print(df.iloc[EXAMPLE_INDEX]['processed_text'])
print(sequences[EXAMPLE_INDEX])

сравнивать ру оглашать победитель ежегодный премия номинация каско год исход весьма закономерный пользователь маркетплейс важно эксперт отдавать vedomoti pre proto bit liderom ingotrah pobedil nominatii kako goda ezhegodnoi premii ravni предпочтение страховой гигант ингосстрах высоко оценивать предлагать клиент арсенал продукт добавлять кредит доверие полюбиться обладатель полис компания мобильный приложение ingomobile благодаря который услуга страховщик доступный буквально движение палец премия сравнивать молодой проводиться второй учитывать репутация сам сравнивать ру обладание приз считаться весьма значимый страховой компания
[4009, 2537, 586, 1977, 839, 931, 1, 4543, 1162, 4010, 1163, 1670, 1534, 1535, 4011, 4544, 3559, 1671, 1164, 336, 459, 1196, 671, 2963, 4545, 1775, 64, 192, 151, 479, 6, 161, 187, 1672, 33, 4012, 839, 4009, 587, 1165, 143, 1595, 480, 4009, 2537, 840, 2538, 4543, 2097, 3559, 64]


In [441]:
x = pad_sequences(sequences, maxlen=MAX_NEWS_LEN)

x

array([[   0,    0,    0, ...,    0,  576,    2],
       [   0,    0,    0, ..., 2097, 3559,   64],
       [   0,    0,    0, ..., 2964, 4546,    2],
       ...,
       [   0,    0,    0, ...,   45,  507,  335],
       [   0,    0,    0, ...,  431,   14,   16],
       [   0,    0,    0, ...,   59,   59, 1592]], dtype=int32)

In [442]:
y = df.drop(['processed_text', 'post_text', 'tokens'], axis=1).values

y 

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [443]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [444]:
ros = RandomOverSampler(random_state=42)

x_train_resampled, y_train_resampled = ros.fit_resample(x_train, y_train)

In [445]:
y_train_resampled

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [ ]:
DIR_DATA_PATH = './data'

In [446]:
with open(f'{DIR_DATA_PATH}/x_train.json', 'w') as f:
    json.dump(x_train_resampled.tolist(), f)

with open(f'{DIR_DATA_PATH}/y_train.json', 'w', encoding='utf-8') as f:
    json.dump(y_train_resampled.tolist(), f, ensure_ascii=False)

In [447]:
with open(f'{DIR_DATA_PATH}/x_test.json', 'w') as f:
    json.dump(x_test.tolist(), f)

with open(f'{DIR_DATA_PATH}/y_test.json', 'w', encoding='utf-8') as f:
    json.dump(y_test.tolist(), f, ensure_ascii=False)